In [32]:
%matplotlib inline
import pandas as pd
import numpy as np
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import psycopg2
from sqlalchemy import create_engine

In [47]:
x_train = pd.read_csv('x_train.csv')
y_train = pd.read_csv('y_train.csv')
x_test = pd.read_csv('x_test.csv')
x_test_add = pd.read_csv('additional_x_files_2EHFvuq/x_test_additional_file.csv')
x_train_add = pd.read_csv('additional_x_files_2EHFvuq/x_train_additional_file.csv')

In [3]:
x_train.head()

,emergency vehicle selection,intervention,alert reason category,alert reason,intervention on public roads,floor,location of the event,longitude intervention,latitude intervention,emergency vehicle,...,delta status preceding selection-selection,departed from its rescue center,longitude before departure,latitude before departure,delta position gps previous departure-departure,GPS tracks departure-presentation,GPS tracks datetime departure-presentation,OSRM response,OSRM estimated distance,OSRM estimated duration
0,5105452,13264186,3,2162,0,0,148.0,2.284796,48.879669,4511,...,2027,1,2.288053,48.884698,NaN,"2.289000,48.885113;2.288861,48.884998;2.288000...",2018-07-08 19:04:43;2018-07-08 19:05:55;2018-0...,"{""code"":""Ok"",""routes"":[{""geometry"":""}yjiH}x}L_...",952.5,105.8
1,4720915,12663715,3,2124,0,1,136.0,2.247464,48.818191,4327,...,28233,1,2.268519,48.823958,NaN,NaN,NaN,"{""code"":""Ok"",""routes"":[{""geometry"":""u|~hHy`zL_...",2238.5,243.2
2,5365374,13675521,3,2163,0,2,139.0,2.258328,48.836759,4509,...,1981,0,2.272944,48.850498,33.0,"2.272972,48.850498;2.269056,48.847443;2.262611...",2018-11-16 10:14:31;2018-11-16 10:15:43;2018-1...,"{""code"":""Ok"",""routes"":[{""geometry"":""_cdiHo}zLm...",3026.4,295.4
3,4741586,12695745,3,2162,0,0,136.0,2.393832,48.776526,5731,...,1842,1,2.388940,48.787732,NaN,"2.394278,48.782112;2.393639,48.776833",2018-01-15 00:42:46;2018-01-15 00:43:58,"{""code"":""Ok"",""routes"":[{""geometry"":""sywhH_pqMy...",1934.2,166.6
4,5381209,13698743,3,2162,0,3,104.0,2.458610,48.875751,5623,...,2716,1,2.439331,48.863814,NaN,"2.439472,48.863724;2.439583,48.863777;2.440917...",2018-11-24 00:37:10;2018-11-24 00:38:22;2018-1...,"{""code"":""Ok"",""routes"":[{""geometry"":""wvfiHgn{Mj...",2706.7,262.6


In [12]:
conn = psycopg2.connect(os.environ['DB_POMPIERS_URL'])
cur = conn.cursor()

cur.execute('CREATE SCHEMA IF NOT EXISTS data')
conn.commit()

In [45]:
def to_table_key(s): return (s.replace(' ','_').replace('-','_')).lower()
def to_exportable_df(df):
    old_cols = df.columns.tolist()
    new_cols = map(lambda x: to_table_key(x), old_cols)
    d_chcols = {k:v for (k,v) in zip(old_cols, new_cols)}
    return d_chcols

def export_df(df, name, schema="data"):
    df = df.rename(columns=to_exportable_df(df))
    engine = create_engine(os.environ['DB_POMPIERS_URL'])
    df.to_sql(name, engine, schema, if_exists='replace', index=False)

In [51]:
for df in ['x_train', 'x_test', 'y_train', 'x_test_add', 'x_train_add']:
    export_df(eval(df), df)